In [10]:
import pandas as pd
import scipy as sc
import numpy as np
import scipy.stats as stats
from scipy.stats import shapiro, kstest, ttest_ind, norm, mannwhitneyu, binom
import math
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.stats.proportion as proportion
import matplotlib.pyplot as plt
import statsmodels.stats.power as smp
from tqdm.auto import tqdm
from random import randint
import seaborn as sns
from itertools import product
from tqdm.notebook import tqdm
import warnings
from pyperclip import paste, copy
import ipywidgets as widgets
from itertools import product
from IPython.display import Markdown as md

%load_ext google.cloud.bigquery
warnings.filterwarnings("ignore")

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pal = [
    "#be0707",
    "#dc3248",
    "#ef5a80",
    "#f982b2",
    "#fca9dd",
    "#ffcfff",
    "#eebdfb",
    "#daacf9",
    "#c19cf8",
    "#a48ef7",
    "#7e82f7",
]

sns.set_theme(style="white", palette=pal, rc={"figure.figsize": (12, 8)})
plt.rcParams["figure.figsize"] = (12, 8)

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [3]:
%%bigquery tables
select *, CONCAT(table_catalog,".",  table_schema, ".", table_name) AS table_ref
FROM `analytics-147612.wh_raw`.INFORMATION_SCHEMA.TABLES
WHERE table_type = "BASE TABLE"

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
tables_list = tables["table_ref"].values

In [3]:
from google.cloud import bigquery

client = bigquery.Client(location="US")

In [12]:
tables["ddl"].iloc[0]

'CREATE TABLE `analytics-147612.wh_raw.copy_trade_log`\n(\n  id INT64,\n  type STRING,\n  date TIMESTAMP,\n  master_account_number INT64,\n  copier_account_number INT64,\n  copier_user_id INT64,\n  commission INT64,\n  amount FLOAT64,\n  deposit_percentage INT64,\n  commission_revshare INT64,\n  commission_type STRING,\n  source STRING,\n  mobile_app STRING,\n  volume_ratio INT64,\n  is_trade_notifications_enabled INT64\n);'

In [16]:
t = []
for table_ref in tables_list:
    # Fetch the table
    table = client.get_table(table_ref)

    # Iterate over the schema and print details
    for field in table.schema:
        if field.field_type == "STRING" and "event" in field.name:
            # print(table_ref)
            # print(f"Field: {field.name}, Type: {field.field_type}, Mode: {field.mode}")
            t.append([table_ref, field.name])

In [17]:
t

[['analytics-147612.wh_raw.webinar_event_bonuses_raw', 'webinar_event'],
 ['analytics-147612.wh_raw.lifetime_events', 'event_type'],
 ['analytics-147612.wh_raw.appsflyer_uninstall_events_report_comoctamarkets',
  'event_name'],
 ['analytics-147612.wh_raw.appsflyer_uninstall_events_report_comoctamarkets',
  'event_value'],
 ['analytics-147612.wh_raw.appsflyer_uninstall_events_report_comoctamarkets',
  'event_revenue'],
 ['analytics-147612.wh_raw.appsflyer_uninstall_events_report_comoctamarkets',
  'event_revenue_currency'],
 ['analytics-147612.wh_raw.appsflyer_uninstall_events_report_comoctamarkets',
  'event_revenue_usd'],
 ['analytics-147612.wh_raw.appsflyer_uninstall_events_report_comoctamarkets',
  'event_source'],
 ['analytics-147612.wh_raw.intercom_chat_errors', 'event_value'],
 ['analytics-147612.wh_raw.intercom_chat_errors', 'event_name'],
 ['analytics-147612.wh_raw.touches_appsflyer', 'event_nature'],
 ['analytics-147612.wh_raw.partner_lifetime_periods_tmp', 'event_type'],
 ['a

In [ ]:
from google.cloud import bigquery

# Initialize a BigQuery client
client = bigquery.Client()

# List of table references in the format `project_id.dataset_id.table_id`
table_references = [
    "your-project-id.your-dataset-id.your-table-id1",
    "your-project-id.your-dataset-id.your-table-id2",
    # Add more table references as needed
]

# Iterate over the table references
for table_ref in table_references:
    # Fetch the table
    table = client.get_table(table_ref)

    # Print the table ID
    print(f"Schema for {table_ref}:")

    # Iterate over the schema and print details
    for field in table.schema:
        print(f"Field: {field.name}, Type: {field.field_type}, Mode: {field.mode}")

    # Add a newline for better readability between tables
    print("\n")

# Note: Replace 'your-project-id', 'your-dataset-id', and 'your-table-id' with

In [8]:
%%bigquery
create or replace table dev_gsokolov.segment as (
SELECT
  u.user_id,
  MIN(u.country) as user_country,
  MIN(ta.trading_account_created_date) AS account_creation_time
FROM
  wh_raw.users AS u
  INNER JOIN amplitude.events_octa_raw_trading_accounts AS ta ON u.user_id = ta.user_id
  AND ta.platform = 'android'
  AND ta.server_name = 'real'
  LEFT JOIN wh_raw.deposits_enhanced AS d ON u.user_id = d.user_id 
  AND ta.trading_account_created_date <= d.created_dt
  AND d.created_dt
  < DATE_ADD(ta.trading_account_created_date, INTERVAL 1 HOUR)
WHERE
  -- d.user_id IS NULL
  DATE(u.registered_dt) BETWEEN '2023-06-01' AND '2024-01-01'
GROUP BY
  1)

Query is running:   0%|          |

""


In [47]:
%%bigquery
WITH
  registeredusers AS (
    SELECT
      user_id,
      country,
      registered_dt
    FROM
      `analytics-147612.wh_raw.users`
    WHERE
      DATE(registered_dt) BETWEEN '2023-06-01' AND '2024-01-01'
      AND country IN (
        'India',
        'Indonesia',
        'Mexico',
        'Brazil',
        'Malaysia'
      )
  ),
  androidusers AS (
    SELECT
      user_id,
      MIN(trading_account_created_date) AS first_trading_account_dt
    FROM
      `analytics-147612.amplitude.events_octa_raw_trading_accounts`
    WHERE
      platform = 'android'
      AND server_name = 'real'
      AND user_id IN (
        SELECT
          user_id
        FROM
          registeredusers
      )
    GROUP BY
      1
  ),
  deposits AS (
    SELECT
      b.user_id as user_id,
      MIN(TIMESTAMP_SECONDS(d.created)) AS first_deposit_time
    FROM
      wh_raw.deposit_requests d
      left join wh_raw.billing_accounts b
      on d.billing_account_id = b.id
      where b.user_id IN (
        SELECT
          user_id
        FROM
          registeredusers
      )
    GROUP BY
      b.user_id
  ),
  filteredusers AS (
    SELECT
      ru.user_id,
      ru.registered_dt,
      ru.country,
      d.first_deposit_time
    FROM
      registeredusers AS ru
      INNER JOIN androidusers AS au ON ru.user_id = au.user_id
      LEFT JOIN deposits AS d ON ru.user_id = d.user_id
    WHERE
      d.user_id IS NULL
      OR TIMESTAMP_DIFF(
        d.first_deposit_time,
        TIMESTAMP(au.first_trading_account_dt),
        HOUR
      ) > 1
  )
SELECT
  *
FROM
  filteredusers;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,registered_dt,country,first_deposit_time
0,32932941,2023-06-11 22:46:53+00:00,Brazil,NaT
1,32936351,2023-06-12 03:23:27+00:00,Brazil,NaT
2,32943594,2023-06-12 11:18:54+00:00,Brazil,NaT
3,32946875,2023-06-12 14:27:46+00:00,Brazil,NaT
4,32948748,2023-06-12 16:23:14+00:00,Brazil,NaT
...,...,...,...,...
1138984,33802505,2023-07-23 00:11:11+00:00,Malaysia,NaT
1138985,33827915,2023-07-24 08:24:12+00:00,Malaysia,NaT
1138986,33720140,2023-07-18 20:59:42+00:00,Malaysia,2023-07-19 11:31:57+00:00
1138987,33758728,2023-07-20 15:54:36+00:00,Malaysia,NaT


In [43]:
s = _42

In [45]:
s["user_id"].nunique()

1138989

In [44]:
s["first_deposit_time"].notna().sum()

36367

In [29]:
from pandas_gbq import to_gbq

segment.to_gbq("dev_gsokolov.segment", "analytics-147612", if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 5745.62it/s]


In [28]:
segment

,user_id,registered_dt,country,first_trading_account_dt
0,35933014,2023-11-01 00:35:29+00:00,India,2023-11-01 00:37:24+00:00
1,35933595,2023-11-01 01:27:00+00:00,India,2023-11-01 01:28:04+00:00
2,35933647,2023-11-01 01:32:43+00:00,India,2023-11-01 01:33:43+00:00
3,35934160,2023-11-01 02:15:04+00:00,India,2023-11-01 02:21:16+00:00
4,35934566,2023-11-01 02:50:10+00:00,India,2023-12-01 07:35:36+00:00
...,...,...,...,...
1102617,32695737,2023-06-01 10:58:33+00:00,Malaysia,2023-06-01 11:01:23+00:00
1102618,32692579,2023-06-01 07:56:01+00:00,Malaysia,2023-06-01 07:57:35+00:00
1102619,32699119,2023-06-01 13:49:06+00:00,Malaysia,2023-06-01 13:51:38+00:00
1102620,32707387,2023-06-01 23:35:18+00:00,Malaysia,2023-06-07 13:28:12+00:00


In [ ]:
from google.cloud import bigquery

# Initialize a BigQuery client
client = bigquery.Client()

# Define the table schema
schema = [
    bigquery.SchemaField("column1", "STRING"),
    bigquery.SchemaField("column2", "INTEGER"),
    bigquery.SchemaField("column3", "FLOAT"),
]

# Define the table reference
table_ref = client.dataset("your_dataset_id").table("your_table_id")

# Define the table object
table = bigquery.Table(table_ref, schema=schema)

# Create the table
client.create_table(table)
%%bigquery
CREATE OR REPLACE TABLE `your-project-id.your-dataset-id.your-table-id` (
  column1 datatype1,
  column2 datatype2,
  ...
)


In [18]:
segment

,user_id,registered_dt,country,first_trading_account_dt
0,35933014,2023-11-01 00:35:29+00:00,India,2023-11-01 00:37:24+00:00
1,35933595,2023-11-01 01:27:00+00:00,India,2023-11-01 01:28:04+00:00
2,35933647,2023-11-01 01:32:43+00:00,India,2023-11-01 01:33:43+00:00
3,35934160,2023-11-01 02:15:04+00:00,India,2023-11-01 02:21:16+00:00
4,35934566,2023-11-01 02:50:10+00:00,India,2023-12-01 07:35:36+00:00
...,...,...,...,...
1102617,32695737,2023-06-01 10:58:33+00:00,Malaysia,2023-06-01 11:01:23+00:00
1102618,32692579,2023-06-01 07:56:01+00:00,Malaysia,2023-06-01 07:57:35+00:00
1102619,32699119,2023-06-01 13:49:06+00:00,Malaysia,2023-06-01 13:51:38+00:00
1102620,32707387,2023-06-01 23:35:18+00:00,Malaysia,2023-06-07 13:28:12+00:00


In [13]:
%%bigquery
SELECT
    table_name,
    search(t.ddl, "payment") AS res
FROM (SELECT
    table_name,
    ddl
FROM wh_raw.INFORMATION_SCHEMA.TABLES) AS t
WHERE search(t.ddl, "payment") = True

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,res
0,payment_transactions,True
1,contests,True
2,cpa_actual_costs,True
3,billing_paysystem_user_restrictions,True
4,ps_report_to_fims_withdrawals_reconciliation,True
5,rebate_payment_error_log,True
6,withdrawal_requests_every_5_minutes,True
7,fraud_flag_every_5_mins,True
8,ps_report_to_fims_deposits_reconciliation,True
9,chargebacks_export_prepared,True


In [19]:
user_count_per_day = segment.groupby(pd.to_datetime(segment["registered_dt"]).dt.date)[
    "user_id"
].count()

user_count_per_day.sort_index()

registered_dt
2023-06-01     4190
2023-06-02     3870
2023-06-03     3600
2023-06-04     3902
2023-06-05     3935
              ...  
2023-12-28    10235
2023-12-29     9795
2023-12-30     8730
2023-12-31     7763
2024-01-01     8296
Name: user_id, Length: 215, dtype: int64

In [20]:
%%bigquery registrations_cnt
SELECT
  registered_dt,
  COUNT(DISTINCT user_id) AS daily_registrations_count
FROM
  (
    SELECT
      DATE (registered_dt) AS registered_dt,
      user_id,
      country
    FROM
      wh_raw.users
  )
WHERE
  registered_dt BETWEEN '2023-06-01' AND '2024-01-01'
  AND country IN (
    'India',
    'Indonesia',
    'Mexico',
    'Brazil',
    'Malaysia'
  )
GROUP BY
  registered_dt
ORDER BY
  registered_dt

Query is running:   0%|          |

Downloading:   0%|          |

In [21]:
registrations_cnt

,registered_dt,daily_registrations_count
0,2023-06-01,12107
1,2023-06-02,11222
2,2023-06-03,10623
3,2023-06-04,11543
4,2023-06-05,12094
...,...,...
210,2023-12-28,17225
211,2023-12-29,16414
212,2023-12-30,14282
213,2023-12-31,13109


In [ ]:
merged_df = pd.merge(
    registrations_cnt, user_count_per_day, left_on="registered_dt", right_index=True
)
merged_df.sort_values(by="registered_dt", ascending=True)

,registered_dt,daily_registrations_count,user_id
0,2023-06-01,12107,4357
1,2023-06-02,11222,4046
2,2023-06-03,10623,3732
3,2023-06-04,11543,4011
4,2023-06-05,12093,4105
...,...,...,...
210,2023-12-28,17225,10436
211,2023-12-29,16414,9999
212,2023-12-30,14282,8881
213,2023-12-31,13109,7895


In [48]:
%%bigquery deposited_cnt
WITH
  depositswithinanhour AS (
SELECT
      b.user_id,
      CASE
        WHEN TIMESTAMP_DIFF (MIN(TIMESTAMP_SECONDS(d.created)), MIN(u.registered_dt), HOUR) <= 1 THEN 1
        ELSE 0
      END AS deposited_within_an_hour
    FROM
      -- wh_raw.deposit_requests d
      -- left join wh_raw.billing_accounts b
      -- on d.billing_account_id = b.id
      -- left join wh_raw.users u on b.user_id =   u.user_id
    WHERE
      TIMESTAMP_SECONDS(d.created) >= u.registered_dt
      AND u.country IN (
        'India',
        'Indonesia',
        'Mexico',
        'Brazil',
        'Malaysia'
      )
    GROUP BY
      b.user_id
  ),
  users_converted AS (
    SELECT
      u.user_id,
      DATE (u.registered_dt) AS registered_dt,
      COALESCE(d.deposited_within_an_hour, 0) AS deposited
    FROM
      wh_raw.users AS u
      LEFT JOIN depositswithinanhour AS d ON u.user_id = d.user_id
      LEFT JOIN amplitude.events_octa_raw_trading_accounts AS ta ON u.user_id = ta.user_id
      AND ta.platform = 'android'
      AND server_name = 'real'
    WHERE
      DATE (u.registered_dt) BETWEEN '2023-06-01' AND '2024-01-01'
      AND u.country IN (
        'India',
        'Indonesia',
        'Mexico',
        'Brazil',
        'Malaysia'
      )
  )
SELECT
  registered_dt,
  SUM(deposited) AS converted_cnt
FROM
  users_converted
GROUP BY
  registered_dt
ORDER BY
  registered_dt ASC

Query is running:   0%|          |

Downloading:   0%|          |

In [49]:
deposited_cnt

,registered_dt,converted_cnt
0,2023-06-01,346
1,2023-06-02,360
2,2023-06-03,226
3,2023-06-04,292
4,2023-06-05,368
...,...,...
210,2023-12-28,534
211,2023-12-29,265
212,2023-12-30,199
213,2023-12-31,147


In [50]:
registrations_cnt

,registered_dt,daily_registrations_count
0,2023-06-01,12107
1,2023-06-02,11222
2,2023-06-03,10623
3,2023-06-04,11543
4,2023-06-05,12094
...,...,...
210,2023-12-28,17225
211,2023-12-29,16414
212,2023-12-30,14282
213,2023-12-31,13109


In [ ]:
merged_df = pd.merge(registrations_cnt, user_count_per_day, on="registered_dt").rename(
    columns={"user_id": "segment_cnt"}
)
# merged_df.rename(columns={'user_id': 'segment_cnt'}, inplace=True)
merged_df

,registered_dt,daily_registrations_count,segment_cnt
0,2023-06-01,12107,4357
1,2023-06-02,11222,4046
2,2023-06-03,10623,3732
3,2023-06-04,11543,4011
4,2023-06-05,12093,4105
...,...,...,...
210,2023-12-28,17225,10436
211,2023-12-29,16414,9999
212,2023-12-30,14282,8881
213,2023-12-31,13109,7895


In [51]:
merged_df = pd.merge(registrations_cnt, user_count_per_day, on="registered_dt").rename(
    columns={"user_id": "segment_cnt"}
)

merged_df = pd.merge(merged_df, deposited_cnt, on="registered_dt")
merged_df = merged_df[
    [
        "registered_dt",
        "daily_registrations_count",
        "segment_cnt",
        "converted_cnt",
    ]
]

In [52]:
%%bigquery overall_conversion
WITH
  registeredusers AS (
    SELECT
      user_id,
      country,
      registered_dt
    FROM
      `analytics-147612.wh_raw.users`
    WHERE
      DATE(registered_dt) BETWEEN '2023-06-01' AND '2024-01-01'
      AND country IN (
        'India',
        'Indonesia',
        'Mexico',
        'Brazil',
        'Malaysia'
      )
  ),
  androidusers AS (
    SELECT
      user_id,
      MIN(trading_account_created_date) AS first_trading_account_dt
    FROM
      `analytics-147612.amplitude.events_octa_raw_trading_accounts`
    WHERE
      user_id IN (
        SELECT
          user_id
        FROM
          registeredusers
      )
    GROUP BY
      1
  ),
  deposits AS (
    SELECT
      b.user_id as user_id,
      MIN(TIMESTAMP_SECONDS(d.created)) AS first_deposit_time
    FROM
      wh_raw.deposit_requests d
      left join wh_raw.billing_accounts b
      on d.billing_account_id = b.id
      where b.user_id IN (
        SELECT
          user_id
        FROM
          registeredusers
      )
    GROUP BY
      b.user_id
  ),
  converted AS (
    SELECT
      ru.user_id,
      ru.registered_dt,
      ru.country,
      au.first_trading_account_dt,
      d.first_deposit_time,
      CASE
        WHEN TIMESTAMP_DIFF(
          d.first_deposit_time,
          au.first_trading_account_dt,
          HOUR
        ) <= 1 THEN 1
        ELSE 0
      END AS deposited_within_an_hour
    FROM
      registeredusers AS ru
      INNER JOIN androidusers AS au ON ru.user_id = au.user_id
      LEFT JOIN deposits AS d ON ru.user_id = d.user_id
  )
SELECT
  *
FROM
  converted
;

Query is running:   0%|          |

Downloading:   0%|          |

In [53]:
p_x = overall_conversion["deposited_within_an_hour"].mean()

In [61]:
0.0196

0.02238543442129132

In [75]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://library.lol/main/658D75E4F3BCE75B68D76A5F126F67D1"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract the book name
    book_name = soup.find("h1").text.strip()
    print(f"Book Name: {book_name}")

    links = soup.find_all("a")
    cloudflare = links[1]["href"]
    ext = cloudflare.split(".")[-1]
    print(ext)
    print(f"Cloudflare Link: {cloudflare}")
else:
    print("Failed to retrieve the webpage")

Book Name: Bayesian Decision Analysis: Principles and Practice
pdf
Cloudflare Link: https://cloudflare-ipfs.com/ipfs/bafykbzacecdqh7adiecjjuse5cvmd6nxfslhmcvujvgueythqexsyjjv5xocg?filename=Jim%20Q.%20Smith%20-%20Bayesian%20Decision%20Analysis_%20Principles%20and%20Practice-Cambridge%20University%20Press%20%282010%29.pdf


In [60]:
import statsmodels.stats.api as sms

# Define parameters
alpha = 0.05
power = 0.8
effect_size = sms.proportion_effectsize(p_x, p_x + 0.00)
sample_size = sms.NormalIndPower().solve_power(
    effect_size, power=power, alpha=alpha, ratio=1.0
)
print(f"Required sample size per group: {int(sample_size)}")

Required sample size per group: 30142


In [56]:
from statsmodels.stats.power import TTestIndPower, NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize, proportions_ztest


def calculate_mde(base_prop, sample_size, power=0.8, alpha=0.05):
    power_analysis = TTestIndPower()
    effect_size = power_analysis.solve_power(
        effect_size=None,
        power=power,
        alpha=alpha,
        nobs1=sample_size,
        ratio=1,
        alternative="two-sided",
    )
    mde = proportion_effectsize(base_prop, base_prop + effect_size)
    return np.abs(mde), effect_size


def calculate_sample_size(base_rate, effect_size, power=0.8, alpha=0.05):
    effect_size = proportion_effectsize(base_rate, base_rate + effect_size)

    sample_size = TTestIndPower().solve_power(
        effect_size=effect_size,
        power=power,
        alpha=alpha,
        ratio=1,
        alternative="two-sided",
    )

    return int(sample_size), math.ceil(sample_size / 100) * 100


p_x, minimum_detectable_effect = (
    overall_conversion["deposited_within_an_hour"].mean(),
    0.004,
)
sample_size, rounded = calculate_sample_size(p_x, minimum_detectable_effect)
print(f"Required sample size per group: {int(sample_size)+1}")

Required sample size per group: 23304


In [ ]:
%%bigquery t
WITH
  android_reg AS (
    SELECT
      user_id,
      touch_name,
      visit_time_dt,
      DATE(visit_time_dt) AS touch_date,
      touch_position,
      c.country,
      device_type,
      os,
      registered_dt,
       -- ROW_NUMBER() OVER (
      -- PARTITION BY
      -- user_id
      -- ORDER BY
      -- visit_time_dt DESC
      -- )                          AS touch_number,
      LAST_VALUE(visit_time_dt) OVER (
        PARTITION BY
          user_id
        ORDER BY
          visit_time_dt DESC ROWS BETWEEN UNBOUNDED PRECEDING
          AND UNBOUNDED FOLLOWING
      ) AS last_touch
    FROM
      wh_reporting.touches_with_user_id t
      LEFT JOIN wh_raw.countries c ON t.country = c.country
    WHERE
      touch_position = 'touch_before_reg'
      AND device_type = 'mobile'
      AND os = 'android'
      AND DATE(registered_dt) BETWEEN '2023-06-01' AND '2024-01-01'
      AND DATE(visit_time_dt) BETWEEN '2023-06-01' AND '2024-01-01'
      AND c.country IN (
        'India',
        'Indonesia',
        'Mexico',
        'Brazil',
        'Malaysia'
      )
  ) -- ,
SELECT
  touch_date,
  COUNT(DISTINCT user_id) AS reg_cnt
FROM
  android_reg
GROUP BY
  touch_date
ORDER BY
  touch_date ASC

Query is running:   0%|          |

Downloading:   0%|          |